In [3]:
#Ту мач рекурсии но что поделать
class Node:
    weight = 0;
    path = '';
    
    def __init__(self, byteWrap, parentNode = None):
        self.left = None; # Пусть левый будет 0, а правый 1
        self.right = byteWrap;
        byteWrap.setParent(self);
        self.parent = parentNode;
        if (parentNode is not None): 
            parentNode.increaseWeight();
            self.path = parentNode.path + '0';
    
    def getWeight(wtf):
        return wtf.weight;
        
    def increaseWeight(wtf):
        wtf.weight += 1;
        if (wtf.left is not None) and (wtf.left is not None) and (wtf.left.getWeight() > wtf.right.getWeight()):
            wtf.left, wtf.right = wtf.right, wtf.left;
            wtf.left.modifyPathAt(len(wtf.path));
            
    def modifyPathAt(wtf, number):
        p = wtf.path[number];
        if (p == '1'): wtf.path = wtf.path[:number] + '0' + wtf.path[number+1:];
        if (p == '0'): wtf.path = wtf.path[:number] + '1' + wtf.path[number+1:];
        if (wtf.left is not None): wtf.left.modifyPath(number);
        if (wtf.right is not None): wtf.right.modifyPath(number);
        
    def insertNode(wtf, node):
        if (wtf.left is None): wtf.left = node;
        node.setPath(wtf.getPath() + '0');
        return node.getPath();
    
    def setPath(wtf, p):
        wtf.path = p;
            
    def getPath(wtf):
        return wtf.path;
    
class ByteWrap:
    def __init__(self):
        self.weight = 1;
        self.parent = None;
        self.path = '';
    
    def getWeight(wtf):
        return wtf.weight;
    
    def increaseWeight(wtf):
        wtf.weight += 1;
        
    def setParent(wtf, node): #Листик в дереве хранит в себе ссылку на родителя, чтобы брать путь из кешового списка
        wtf.parent = node;
        wtf.parent.increaseWeight();
        wtf.path = node.path + '1';
        
    def getPath(wtf):
        return wtf.path;
    
    def modifyPathAt(wtf, number):
        p = wtf.path[number];
        if (p == '1'): wtf.path = wtf.path[:number] + '0' + wtf.path[number+1:];
        if (p == '0'): wtf.path = wtf.path[:number] + '1' + wtf.path[number+1:];

In [17]:
def compress(srcName):
    resultBits = '';
    with open(srcName, 'rb') as src:
        # Кэшим байтики
        listUsedBytes = [None for i in range(256)];

        file = bytearray(src.read());
        if (file[0] == ''): raise ValueError('Файл пуст');

        # Оборачиваем стартовый байтик
        wrap = ByteWrap();
        listUsedBytes[file[0]] = wrap;

        # Инициализируем голову и ноду с пустым листом
        treeHead = Node(wrap);
        nodeWithEmptyLeft = treeHead;

        #Результирующая строка с байтами
        resultBits = format(file[0], '08b') ;

        total = 0;
        for byte in file:
            total += 1;
            if (total % 1000_000 == 0): print(total // 1_000_000, 'Мбайт' );
            if (listUsedBytes[byte] is not None):
                resultBits += listUsedBytes[byte].getPath();
                listUsedBytes[byte].increaseWeight();
            else:
                listUsedBytes[byte] = ByteWrap();
                newNode = Node(listUsedBytes[byte], nodeWithEmptyLeft);
                nodeWithEmptyLeft.insertNode(newNode);
                nodeWithEmptyLeft = newNode;
                resultBits += newNode.path + format(byte, '08b');

    newBytes = [resultBits[i:i+8] for i in range(0, len(resultBits), 8)]; # Я не дружу с битами и питон тоже
    newBytesData = bytes([int(bits, 2) for bits in newBytes]);

    with open(srcName + '.pupupu', 'wb') as dest:
        dest.write(newBytesData); 
        print('Всё');
    # Несмотря на то, что дурында эта памяти не жрет, работает крайне долго
    
def decompress(srcName):
    resultBits = '';
    with open(srcName, 'rb') as src:
        listUsedBytes = [None for i in range(256)];
        data = bytearray(src.read());
        bitString = ''.join(format(byte, '08b') for byte in data);
        
        firstByte = bitString[:8];
        resultBits += firstByte;
        
        wrap = ByteWrap();
        listUsedBytes[int(firstByte, 2)] = wrap;
        
        treeHead = Node(wrap);
        nodeWithEmptyLeft = treeHead;
        currentNode = treeHead;
        
        pointer = 8;
        while(pointer < len(bitString)):
            if (pointer % 100_000 == 0): print(pointer * 8 // len(bitString));
            if (type(currentNode) is Node):
                symb = bitString[pointer];
                if (symb == '0'): currentNode = currentNode.left;
                if (symb == '1'): currentNode = currentNode.right;
            if (currentNode is None):
                byte = bitString[pointer + 1 : pointer + 9];
                wrap = ByteWrap();
                listUsedBytes[int(byte, 2)] = wrap;
                newNode = Node(wrap);
                nodeWithEmptyLeft.insertNode(newNode);
                nodeWithEmptyLeft = newNode;
                pointer += 8;
                resultBits += byte;
            if (type(currentNode) is ByteWrap):
                currentNode = treeHead;
                for i in range(0, len(listUsedBytes)):
                    if (listUsedBytes[i] == currentNode):
                        resultBits += format(number, '08b');
                        break;
            pointer += 1;
            
    newBytes = [resultBits[i:i+8] for i in range(0, len(resultBits), 8)];
    newBytesData = bytes([int(bits, 2) for bits in newBytes]);
    with open('decompressed' + srcName.replace('.pupupu', ''), 'wb') as dest:
        dest.write(newBytesData); 
        print('Всё');

In [ ]:
compress('example.mp4');

In [15]:
decompress('example.mp4.pupupu');

0
Всё
